In [ ]:
l = list(range(1, 12))

In [ ]:
n = 3

In [ ]:
[l[i:i + n] for i in range(0, len(l), n)]

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# 1 - Dataset creation

CSV

In [ ]:
import os
import glob
import pandas as pd

In [ ]:
# dataset_path = '/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/'

In [ ]:
# base_dir = '/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/'
# experiences = os.listdir(base_dir)
# all_image_path = []
# for experience in experiences:
#     folder_path = os.path.join(base_dir, experience)
#     if not os.path.isdir(folder_path):
#         continue
#     if "rectified" not in folder_path:
#         continue
#     if "reference" in folder_path:
#         continue
#     print(folder_path)
#     all_image_path += glob.glob(folder_path + '/*.jpg')

In [ ]:
# all_image_path = glob.glob('/root/data/depthmap/*.jpg')
# all_image_path = sorted(all_image_path, key = lambda k:int(os.path.basename(k).split('.')[0].split('_')[-1]))

In [ ]:
all_image_path = []
with open('/root/thomas/github/cv_research/thomas/full_pipeline/small_pen/to_forwardpass.txt', 'r') as f:
    for line in f:
        all_image_path.append(line[:-1])
all_image_path = sorted(all_image_path)

In [ ]:
dataframe = pd.DataFrame(all_image_path[:], columns=['image path'])
dataframe.head()

Data Loader

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from PIL import Image
import skimage

In [ ]:
class DataGenerator(Dataset):
    """
    Load a dataset for target estimation
    
    Args:
       - dataframe: DataFrame of image path
    """
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __getitem__(self, index):
        img_path = self.dataframe.iloc[:, 0][index]
        img = skimage.io.imread(img_path)
        
        return img, img_path
    
    def __len__(self):
        return len(self.dataframe)

In [ ]:
data_generator = DataGenerator(dataframe)

# 2 - MaskRCNN forward pass

### 2.1 - Load the model

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import keras

sys.path.insert(0, '../libraries')
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(WEIGHTS_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
image_size = 1024
rpn_anchor_template = (1, 2, 4, 8, 16) # anchor sizes in pixels
rpn_anchor_scales = tuple(i * (image_size // 16) for i in rpn_anchor_template)

class FishConfig(Config):
    """Configuration for training on the shapes dataset.
    """
    # name your experiments here
    NAME = "full"

    # Train on 1 GPU and 2 images per GPU. Put multiple images on each
    # GPU if the images are small. Batch size is 2 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 7 + 1  # background + 3 shapes (triangles, circles, and squares)

    # Use smaller images for faster training. 
    IMAGE_MAX_DIM = image_size
    IMAGE_MIN_DIM = image_size
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = rpn_anchor_scales

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    STEPS_PER_EPOCH = 1000

    VALIDATION_STEPS = 300
    
config = FishConfig()
config.display()

In [ ]:
# model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
model_path = '/root/data/models/small_pen/body_parts/logs/body_parts_uncomlpete_1024_20181010_21H08/mask_rcnn_body_parts_uncomlpete_1024_0100.h5'

In [ ]:
class InferenceConfig(FishConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"

# model_path = '/root/data/models/erko/mask_rcnn_instance_segmentation/logs/full_20181002_19H09/mask_rcnn_full_0097.h5'
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# import time

In [ ]:
original_image, img_path = data_generator[0]

In [ ]:
12 % 5

In [ ]:
# speed = []
# for i in range(1, 10):
#     class InferenceConfig(FishConfig):
#         GPU_COUNT = 1
#         print(i)
#         IMAGES_PER_GPU = i

#     inference_config = InferenceConfig()

#     # Recreate the model in inference mode
#     model = modellib.MaskRCNN(mode="inference", 
#                               config=inference_config,
#                               model_dir=MODEL_DIR)

#     # Get path to saved weights
#     # Either set a specific path or find last trained weights
#     # model_path = os.path.join(ROOT_DIR, ".h5 file name here")
#     model.find_last()[1]

#     # Load trained weights (fill in path to trained weights here)
#     assert model_path != "", "Provide path to trained weights"

#     # model_path = '/root/data/models/erko/mask_rcnn_instance_segmentation/logs/full_20181002_19H09/mask_rcnn_full_0097.h5'
#     print("Loading weights from ", model_path)
#     model.load_weights(model_path, by_name=True)

#     start = time.time()
#     results = model.detect([original_image]*i, verbose=1)
#     end = time.time()
#     speed.append(end - start)
    

In [ ]:
# linear = [k*speed[0] for k in range(1, 10)]

In [ ]:
# plt.plot(range(1, 10), linear)
# plt.plot(range(1, 10), speed)
# plt.legend(['ohne batch', 'mit batch'])

### 2.2 - Visualize results

In [ ]:
# try:
#     while True:
#         for image_id in range(len(data_generator)):
#             original_image, _ = data_generator[image_id]
#             results = model.detect([original_image], verbose=1)
#             r = results[0]
#             visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#                             ['BG', 'salmon'], r['scores'], ax=get_ax())

#             input('Press enter to continue: ')
# except KeyboardInterrupt:
#     pass

### 2.3 - Save coco file

In [ ]:
import json
import pylab
import matplotlib.pyplot as plt
from tempfile import NamedTemporaryFile
from pycocotools.coco import COCO
from mextra.utils import result_to_coco
from cococreatortools import * 
from datetime import datetime

In [ ]:
from tqdm import tqdm

In [ ]:
INFO = {
    "description": "Fish data",
    "url": "https://github.com/waspinator/pycococreator",
    "version": "0.1.0",
    "year": 2018,
    "contributor": "thossler",
    "date_created": datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "Attribution-NonCommercial-ShareAlike License",
        "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
    }
]

CATEGORIES = [
    {
        'id': 1,
        'name': 'salmon',
        'supercategory': 'fish',
    }
]

In [ ]:
coco_output = {"info": INFO, "licenses": LICENSES, "categories": CATEGORIES, "images": [], "annotations": []}

In [ ]:
# len(data_generator)
segmentation_id = 1

In [ ]:
for image_id in tqdm(range(len(data_generator))):
    original_image, img_path = data_generator[image_id]
    height,width,_ = original_image.shape
    print(img_path)
    if original_image is not None:
        print(image_id)
        
        results = model.detect([original_image], verbose=1)[0]

        # create the coco stuff
        image_info = create_image_info(image_id, img_path, [width, height])
        coco_output['images'].append(image_info)

        # loop through all the fish detected
        detections_number = len(results['class_ids'])
        for k in range(detections_number):
            # print(segmentation_id)
            category_info = {'id': int(results['class_ids'][k])}
            binary_mask = results['masks'][..., k]
            annotation_info = create_annotation_info(segmentation_id, image_id, category_info, binary_mask,
                                                     [width, height], bounding_box=results['rois'][k, ...], tolerance=2)
            if annotation_info is not None:
                coco_output["annotations"].append(annotation_info)
            segmentation_id += 1


In [ ]:
results['class_ids']

In [ ]:
image_info

In [ ]:
model.config.BATCH_SIZE

In [ ]:
config.BATCH_SIZE

In [ ]:
with open('/root/data/small_pen_data_collection/body_parts_detection_20181017.json', 'w') as f:
    json.dump(coco_output, f)